In [41]:
from utils.get_news import get_stock_news
import datetime as dt
from utils.data_preparation import get_scores,prepare_price,prepare_mix_data_lstm

from pyspark.ml.evaluation import MulticlassClassificationEvaluator
import sparknlp
from pyspark.ml.classification import LinearSVC,LinearSVCModel
from utils.split import split_data_for_ml
from pyspark.sql.types import DoubleType,IntegerType
import sparknlp
# Load the model
import warnings
from pyspark.sql.types import *
from pyspark.sql.functions import col,max
from pyspark.sql.functions import when
import datetime as dt


# Ignore all warnings
warnings.filterwarnings("ignore")

In [2]:
spark = sparknlp.start()

24/05/15 17:24:11 WARN Utils: Your hostname, alber-victus resolves to a loopback address: 127.0.1.1; using 10.251.208.55 instead (on interface wlp4s0)
24/05/15 17:24:11 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/alber/.ivy2/cache
The jars for the packages stored in: /home/alber/.ivy2/jars
com.johnsnowlabs.nlp#spark-nlp_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-1b5b0316-5f5c-4b94-9835-02828459ad62;1.0
	confs: [default]
	found com.johnsnowlabs.nlp#spark-nlp_2.12;5.3.3 in central
	found com.typesafe#config;1.4.2 in central
	found org.rocksdb#rocksdbjni;6.29.5 in central
	found com.amazonaws#aws-java-sdk-s3;1.12.500 in central
	found com.amazonaws#aws-java-sdk-kms;1.12.500 in central
	found com.amazonaws#aws-java-sdk-core;1.12.500 in central
	found commons-logging#commons-logging;1.1.3 in central
	found commons-codec#commons-codec;1.15 in central
	found org.apache.httpcomponents#httpclient;4.5.13 in central
	found org.apache.httpcomponents#httpcore;4.4.13 in central
	found software.amazon.ion#ion-java;1.0.2 in central
	found joda-time#joda-time;2.8.1 in central
	found com.amazonaws#jmespath-java;1.12.500 in central


In [3]:
stock = "NVIDIA"
path_to_save = "../data/json/demo/"

In [4]:
# get_stock_news(stock, 
#                    dir_to_save=path_to_save,
#                    end_date='today',
#                    start_year=2024, start_month=5, start_day=3,
#                    day_range=2)
    

In [5]:
scores = get_scores(stock='demo',spark=spark)
scores.show()

+----------+---------------+------------------+
|      date|afinn_sentiment|     pnn_sentiment|
+----------+---------------+------------------+
|2024-05-03|      3.5799918|0.8305084745762712|
|2024-05-04|      3.2656653|               1.0|
|2024-05-05|        3.76348|               0.9|
|2024-05-06|      2.7413003|0.9333333333333333|
|2024-05-07|      2.0712945|0.9130434782608695|
|2024-05-08|            0.0|               1.0|
|2024-05-09|      1.7120807|0.9393939393939394|
|2024-05-10|      2.3216608|0.9047619047619048|
|2024-05-11|      1.5797324|               1.0|
|2024-05-12|      2.5850167|0.8181818181818182|
|2024-05-13|      1.6851234| 0.926829268292683|
|2024-05-14|       1.712483|               1.0|
|2024-05-15|      2.5953417|0.9393939393939394|
+----------+---------------+------------------+



In [6]:
df = prepare_mix_data_lstm(scores,stock='NVIDIA',spark=spark)
df.show()

+----------+---------------+------------------+--------------------+---------------------+-------------------------------------+
|      date|afinn_sentiment|     pnn_sentiment|price_percent_change|volume_percent_change|next_day_price_percent_change_shifted|
+----------+---------------+------------------+--------------------+---------------------+-------------------------------------+
|2024-05-06|      2.7413003|0.9333333333333333|           3.7741172|             -5.55755|                           -1.7212987|
|2024-05-07|      2.0712945|0.9130434782608695|          -1.7212987|            16.251598|                          -0.15681063|
|2024-05-08|            0.0|               1.0|         -0.15681063|           -25.522589|                           -1.8415724|
|2024-05-09|      1.7120807|0.9393939393939394|          -1.8415724|             16.05423|                             1.274416|
|2024-05-10|      2.3216608|0.9047619047619048|            1.274416|           -11.292733|       

In [40]:
last_date = df.agg(max(col("date")).alias("date")).take(1)[0].date

In [43]:
last_date

datetime.date(2024, 5, 14)

In [54]:
predicted_day = last_date.day + 1
predicted_month = last_date.month
predicted_month = last_date.strftime("%B")

In [8]:
model_path_false = "../models/_binary_smvNVIDIA_emotion_False/"
model_path_true = "../models/_binary_smvNVIDIA_emotion_True/"

In [55]:
def bulish_bearish(model_path,emotion):

    
    data = split_data_for_ml(df=df,emotion=emotion,classification=True,spark=spark)

    loaded_model = LinearSVCModel.load(model_path)
    predictions = loaded_model.transform(data)

    pred = predictions.tail(1)[0].prediction


    if emotion:
        key = "without"
    else:
        key = "with"

    if pred == 1:

        trend = "bullish"
    else:
        trend = "bearish"
        
    print(f"\nSVM {key} sentiment scores predicts {trend} market for Nvidia stock for {day} of {predicted_month}!!\n")

    

In [ ]:
for path,emotion in zip([model_path_false,model_path_true],[False,True]):

    
    bulish_bearish(path,emotion)


In [28]:
spark.stop()